In [1]:
import torch
from datasets import load_dataset
# from pyvi import ViTokenizer, ViPosTagger
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from datasets import Dataset,DatasetDict
import torch.nn as nn
import pandas as pd 
SEED = 1337


In [2]:
df_train = load_dataset("json" , data_files = '/kaggle/input/uit-full-ranking-data/FULL_37K_RANKING_6_DATA' )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-870040edc48df203/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
df_train = df_train["train"].to_pandas ()

In [4]:
df_train

,context,claim,verdict,evidence,domain,__index_level_0__,new_context,labels
0,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...",giao-duc,7125,công ty cũng có chế độ đãi ngộ tốt có lương th...,2
1,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...",giao-duc,7126,bên cạnh đó faba thường xuyên tổ chức những dự...,2
2,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...",giao-duc,7127,bên cạnh đó faba thường xuyên tổ chức những dự...,2
3,"Đối với các nhà khoa học, trí thức, điều kiện ...",Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",giao-duc,7128,không riêng kỹ sư phần mềm đối với bất kỳ lĩnh...,2
4,"Đối với các nhà khoa học, trí thức, điều kiện ...","Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...",giao-duc,7129,theo adeco việt nam mức lương cao nhất đối với...,2
...,...,...,...,...,...,...,...,...
37962,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...",Thái Lan không dự kiến đón ít nhất 5 triệu khá...,REFUTED,Thái Lan dự kiến đón ít nhất 5 triệu khách Tru...,the-gioi,35077,thái lan dự kiến đón ít nhất 5 triệu khách tru...,0
37963,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...","Trước đại dịch, người Trung Quốc chiếm gần 1/2...",REFUTED,"Trước đại dịch, người Trung Quốc chiếm gần 1/3...",the-gioi,35078,trước đại dịch người trung quốc chiếm gần 1/3 ...,0
37964,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...",Zhu nói rằng Số tiền 290.000 USD không thể mua...,REFUTED,"""Số tiền đó chỉ có thể mua được căn hộ nhỏ ở v...",the-gioi,35079,số tiền đó chỉ có thể mua được căn hộ nhỏ ở vị...,0
37965,"""Tôi không có đầu óc đầu tư chứng khoán và đan...","Tiền tiết kiệm của các hộ gia đình giảm 9,9 ng...",REFUTED,"Tiền tiết kiệm của các hộ gia đình tăng 9,9 ng...",the-gioi,35080,tiền tiết kiệm của các hộ gia đình tăng 99 ngh...,0


In [16]:
n_folds = 5
from sklearn.model_selection import StratifiedGroupKFold
cv = StratifiedGroupKFold(n_splits=n_folds)

In [22]:
for i , (train_idx , val_idx ) in enumerate ( cv.split (df_train.index , df_train.verdict , df_train.context) ) :
    train = df_train.iloc [train_idx]
    val = df_train.iloc[val_idx]
    train.to_json ("TRAIN CONTEXT SPLIT {}".format (i + 1))
    val.to_json ("Validation CONTEXT SPLIT {}".format (i + 1))